<a href="https://colab.research.google.com/github/carvalheirafc/deep_learning_stuff/blob/master/neural_networks/le_net/le_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Section


In [0]:
from keras.preprocessing.image import img_to_array
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers.convolutional import Convolution2D
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.utils import np_utils
from keras import backend
#from google.colab import drive
from imutils import paths
import cv2
import numpy as np
import os
import tensorflow as tf

Using TensorFlow backend.


## LeNet Class Definition


In [0]:
class LeNet:
  @staticmethod
  def build(n_channels, 
            rows, 
            cols, 
            n_classes, 
            activation='relu', 
            weights_path=None):
    model = Sequential()
    
    input_Shape = (rows, cols, n_channels)
    
    # Convolution >> Activation >> Pool of Layers
    model.add(Conv2D(20, 5, padding='same', input_shape=input_Shape))
    model.add(Conv2D(20, kernel_size=5, activation=activation, input_shape=(rows, cols, n_channels)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Conv2D(50, 5, activation=activation))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(500))
    model.add(Activation(activation))
    
    model.add(Dense(n_classes))
    model.add(Activation('softmax'))
    
    # if a weights path is supplied (inicating that the model was
    # pre-trained), then load the weights
    if weights_path is not None:
      model.load_weights(weights_path)
    
    return model

## Reading or Import Image Files.


In [0]:
#drive.mount('/content/gdrive')

In [0]:
root_path = 'Letras/'

data = []
labels = []
all_images_paths = sorted(list(paths.list_images(root_path)))
current = -1

try:
    for image_path in all_images_paths:
    
    
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = img_to_array(image)
        data.append(image)
    
        label = image_path.split(os.path.sep)[-2][7:]
        label = int(label)
        labels.append(label)
    
        if current != label:
            print('Loading Files into arrays...')
            print('Loading Class [{}]'.format(label))
        current = label

    print('All Files are Loaded Successfully...')

except IOError:
    print('Something went Wrong loading the files')    

Loading Files into arrays...
Loading Class [0]
Loading Files into arrays...
Loading Class [10]
Loading Files into arrays...
Loading Class [11]
Loading Files into arrays...
Loading Class [12]
Loading Files into arrays...
Loading Class [13]
Loading Files into arrays...
Loading Class [14]
Loading Files into arrays...
Loading Class [15]
Loading Files into arrays...
Loading Class [16]
Loading Files into arrays...
Loading Class [17]
Loading Files into arrays...
Loading Class [18]
Loading Files into arrays...
Loading Class [19]
Loading Files into arrays...
Loading Class [1]
Loading Files into arrays...
Loading Class [20]
Loading Files into arrays...
Loading Class [21]
Loading Files into arrays...
Loading Class [22]
Loading Files into arrays...
Loading Class [23]
Loading Files into arrays...
Loading Class [24]
Loading Files into arrays...
Loading Class [25]
Loading Files into arrays...
Loading Class [2]
Loading Files into arrays...
Loading Class [3]
Loading Files into arrays...
Loading Class [

In [0]:
save_files = False

data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

if save_files:
    try:
        np.save('char_input_data', data)
        np.save('labels', labels)
    except IOError:
        print('Error while saving the files')

## Model Configuration and Train Ru

- Epochs: 10

- Learning Rate: 1e-3

- Batch Size: 32

In [0]:
config = tf.ConfigProto()
sess = tf.Session(config=config) 
backend.set_session(sess)

In [0]:
dlabels = np_utils.to_categorical(labels, 26)

n_epochs = 10
ini_learning_rate = 1e-3
batch_size = 32

model = LeNet.build(n_channels=1, 
                    rows=28, 
                    cols=28, 
                    n_classes=26, 
                    activation='relu', 
                    weights_path=None)

opt = Adam(lr=ini_learning_rate, decay=ini_learning_rate / n_epochs)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

H = model.fit(x=data, 
              y=dlabels, 
              batch_size=batch_size, 
              epochs=n_epochs, 
              verbose=1)

Epoch 1/10
20928/20928 [==============================] - 5s 242us/step - loss: 0.0257 - acc: 0.9919
Epoch 2/10
20928/20928 [==============================] - 3s 163us/step - loss: 0.0082 - acc: 0.9972
Epoch 3/10
20928/20928 [==============================] - 3s 166us/step - loss: 0.0060 - acc: 0.9980
Epoch 4/10
20928/20928 [==============================] - 3s 165us/step - loss: 0.0048 - acc: 0.9983
Epoch 5/10
20928/20928 [==============================] - 3s 164us/step - loss: 0.0041 - acc: 0.9986
Epoch 6/10
20928/20928 [==============================] - 3s 164us/step - loss: 0.0030 - acc: 0.9989
Epoch 7/10
20928/20928 [==============================] - 3s 164us/step - loss: 0.0031 - acc: 0.9989
Epoch 8/10
20928/20928 [==============================] - 3s 165us/step - loss: 0.0025 - acc: 0.9991
Epoch 9/10
20928/20928 [==============================] - 3s 165us/step - loss: 0.0025 - acc: 0.9992
Epoch 10/10
20928/20928 [==============================] - 3s 166us/step - loss: 0.0017 - a

## Saving the Model Weights

In [0]:
weights_file = 'weights_Letras.h5'
try:
    model.save_weights(weights_file)
    print('File Writed as: {}'.format(weights_file))
except IOError:
    print('Error while saving the Model weights')



File Writed as: weights_Letras.h5
